## Screening tasks - Task 2

In [17]:
import numpy as np
import matplotlib.pyplot as plt

import qiskit
from qiskit.visualization import *
from qiskit import *
from math import pi

In [18]:
from qiskit.providers.aer import noise
from qiskit.providers.ibmq import least_busy

### Set up IBMQ with noise and noiseless backend for testing

In [19]:
IBM_TOKEN = 'YOUR IBM TOKEN HERE'

In [22]:
IBMQ.save_account(IBM_TOKEN,
                  overwrite=True)

provider = IBMQ.load_account()
try:
    least_busy_device = least_busy(provider.backends(simulator=False, 
                                                     filters=lambda b: b.configuration().n_qubits > 2))
except:
    print("All devices are currently unavailable.")
    lbd = str(least_busy_device)
    print("Running on current least busy device: ", lbd)
noise_backend = least_busy_device
nonoise_backend = Aer.get_backend("qasm_simulator")

/Users/juriegermishuys/anaconda3/envs/qnlp/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2020-09-13 21:44:47,311: Credentials are already in use. The existing account in the session will be replaced.


### Create a Qiskit circuit that returns the required state

In [23]:
qc = QuantumCircuit(2)
# Apply H-gate to the first:
qc.h(0)
# Apply a CNOT:
qc.cx(0,1)
# Apply a NOT to the second:
qc.x(1)
qc.draw()

┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐┌───┐
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

In [24]:
# Let's see the result:
backend = Aer.get_backend('statevector_simulator')
final_state = execute(qc,backend).result().get_statevector()
# Print the statevector neatly:
print(final_state)

[0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]


### Equivalent circuit using only CNOT, Rx, Ry

In [25]:
qc = QuantumCircuit(2)
# Apply H-gate to the first:
qc.rx(pi, 0)
qc.ry(-pi/2, 0)
# Apply a CNOT:
qc.cx(0,1)
qc.rx(-pi, 1)
qc.draw()

┌────────┐┌───────────┐                
q_0: ┤ RX(pi) ├┤ RY(-pi/2) ├──■─────────────
     └────────┘└───────────┘┌─┴─┐┌─────────┐
q_1: ───────────────────────┤ X ├┤ RX(-pi) ├
                            └───┘└─────────┘

In [26]:
# Let's see the result:
backend = Aer.get_backend('statevector_simulator')
final_state = execute(qc,backend).result().get_statevector()
# Print the statevector neatly:
print(final_state)

[ 4.32978028e-17+0.j          4.32978028e-17+0.70710678j
 -4.32978028e-17+0.70710678j  4.32978028e-17+0.j        ]


### Optimise using gradient descent

#### Start off with the ideal (target) distribution across states (01, 10)

In [27]:
import numpy as np
target_distr = np.array([0., 0., 0.5, 0.5])

#### Ensure that the order extracted remains fixed throughout

In [28]:
qubit_order = {'11': 0, '00': 1, '01': 2, '10': 3}

#### Create circuit

In [29]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qc = QuantumCircuit(2)
    # Apply H-gate to the first:
    qc.rx(params[0], 0)
    qc.ry(params[1], 0)
    # Apply a CNOT:
    qc.cx(0,1)
    qc.rx(params[2], 1)
    qc.measure_all()
    return qc

#### Optimise the objective function which looks to match the output distribution from the quantum circuit to the target distribution

In [31]:
from qiskit import Aer, execute
from qiskit.aqua.components.optimizers import COBYLA, AQGD, SPSA
backend = nonoise_backend

# Set number of trials and random initialisation for each parameters
NUM_SHOTS = 1000
params = np.random.rand(3)

def get_probability_distribution(counts):
    for k in qubit_order.keys():
        if k not in counts:
            counts[k] = 0
    output_distr = [counts['11'] / NUM_SHOTS, counts['00'] / NUM_SHOTS, 
                    counts['01']/ NUM_SHOTS, counts['10'] / NUM_SHOTS]
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    result = execute(qc, backend, shots=NUM_SHOTS).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(4)])
    return cost



In [32]:
def optimise(init_params=params, objective_function=objective_function, 
             optimizer=AQGD(maxiter=1000, tol=0.0001), shots=NUM_SHOTS, backend=backend):
    
    # Create the initial parameters (noting that our TWO qubit variational form has 3 parameters)
    ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)
    # Obtain the output distribution using the final parameters
    qc = get_var_form(ret[0])
    counts = execute(qc, backend, shots=shots).result().get_counts(qc)
    output_distr = get_probability_distribution(counts)
    
    return {"target": target_distr, "obtained": output_distr,
            "error (manhattan distance)": ret[1], "parameters": ret[0]}

In [36]:
experiments = {}
for n in [1, 10, 100, 1000]:
    NUM_SHOTS = n
    experiments[n] = optimise()

In [37]:
experiments

{1: {'target': array([0. , 0. , 0.5, 0.5]),
  'obtained': [233.0, 125.0, 416.0, 226.0],
  'error (manhattan distance)': 1.0,
  'parameters': array([ 2.11047416, -1.03081151,  4.35849963])},
 10: {'target': array([0. , 0. , 0.5, 0.5]),
  'obtained': [3.5, 2.9, 44.0, 49.6],
  'error (manhattan distance)': 0.39999999999999997,
  'parameters': array([-0.27943673,  1.53323268,  3.64252063])},
 100: {'target': array([0. , 0. , 0.5, 0.5]),
  'obtained': [0.25, 0.26, 4.34, 5.15],
  'error (manhattan distance)': 0.0,
  'parameters': array([-4.63488345,  3.68459442,  2.73572427])},
 1000: {'target': array([0. , 0. , 0.5, 0.5]),
  'obtained': [0.0, 0.0, 0.804, 0.196],
  'error (manhattan distance)': 0.0,
  'parameters': array([-6.53484491, -8.57511327,  3.13337353])}}

It is clear from these experiments that as we increase the number of measurements, our measurements become more accurate and do not suffer so much due to the noise. (These experiments were reset in terms of noiseless backends so that the notebook could be run without using an IBMQ token, but it can easily be re-run with the actual quantum backends). 

### Test solution

In [92]:
qc = QuantumCircuit(2)
# Apply H-gate to the first:
qc.rx(experiments[100]['parameters'][0], 0)
qc.ry(experiments[100]['parameters'][1], 0)
# Apply a CNOT:
qc.cx(0,1)
qc.rx(experiments[100]['parameters'][2], 1)
qc.draw()

┌───────────┐┌─────────────┐                   
q_0: ┤ RX(7.834) ├┤ RY(-8.2262) ├──■────────────────
     └───────────┘└─────────────┘┌─┴─┐┌────────────┐
q_1: ────────────────────────────┤ X ├┤ RX(3.1219) ├
                                 └───┘└────────────┘

In [93]:
# Let's see the result:
backend = Aer.get_backend('statevector_simulator')
final_state = execute(qc,backend, shots=10000).result().get_statevector()
final_counts = execute(qc,backend, shots=10000).result().get_counts(qc)
# Print the statevector neatly:
print("Final state vector", final_state)
print("Final state distribution", final_counts)

Final state vector [ 6.94854796e-03+0.j         -7.09510167e-01+0.01318666j
  4.31383536e-17-0.70450278j -1.30060705e-04-0.00699794j]
Final state distribution {'00': 4.8282318693e-05, '01': 0.503578565580366, '10': 0.496324164075472, '11': 4.8988025468e-05}


Our optimised parameters achieve the desired outcome between states '01' and '10'. 

### Bonus Question

How to make sure you produce state  |01⟩  +  |10⟩  and not any other combination of |01> + e(i*phi)|10⟩ (for example |01⟩  -  |10⟩)?

Attempt:
    
    This places a restriction on the values of phi, i.e. e(i*phi) = 1. This restricts phi to multiples of 2*pi 
    Thus if we restrict our parameters to these values we will avoid these combinations.

In [108]:
# End of Notebook